In [1]:
import imp
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

C:\Users\TAM\AppData\Local\Temp\ipykernel_19992\2202898749.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
# INPUT 
data_input=pd.read_excel("../VACP/Python NCC Min/Analysing Qty 4 Months/02 - Back Order 2022 VAPC - 2022 24 08 - CHECKED.xlsx")

In [3]:
# NHAP - XUAT - TON  * GIA BAN * DANH MUC HANG HOA

data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5)
data_nhapxuatton=data_nhapxuatton.rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" }).dropna(subset=['ID']).reset_index(drop=True)


data_banhang=pd.read_excel("../VACP/Bao cao Ban hang Power BI/Sổ chi tiết bán hàng.xls",usecols="A,C:D,F,H,K:M",skipfooter=1,header=6)
data_banhang=data_banhang.rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Giá bán",
                                "Giá bán":"Giá bán giảm"
                                }).dropna(subset=['ID']).reset_index(drop=True)
                                
data_giaban=pd.read_excel("../VACP/Python NCC Min\Báo cáo giá bán, tồn kho.xls",usecols="C,F",skipfooter=1,header=4)
data_giaban=data_giaban.rename(columns={"Mã số":"ID"}).dropna(subset=['ID']).reset_index(drop=True)

data_danhmuchanghoa=pd.read_excel("../VACP/Python NCC Min/Danh mục vật tư hàng hóa .xls",usecols="A,C:D,M")
data_danhmuchanghoa=data_danhmuchanghoa.rename(columns={
                                "Unnamed: 0":"ID Nhóm hàng",
                                "Unnamed: 2":"ID",
                                "Unnamed: 3":"Tên vật tư",
                                "Unnamed: 12":"Hãng"
                                }).dropna(subset=['ID','ID Nhóm hàng']).reset_index(drop=True)



C:\Users\TAM\AppData\Local\Temp\ipykernel_19992\773044771.py:3: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5)


In [4]:
# Giá vốn
TransCost=0.05
data_nhapxuatton['Giá vốn'] =(1+TransCost)*np.where(data_nhapxuatton["Giá trị xuất"]>0,
                                        data_nhapxuatton["Giá trị xuất"]/data_nhapxuatton["Tổng xuất"],
                                            np.where(data_nhapxuatton["Giá trị tồn"]>0,
                                                data_nhapxuatton["Giá trị tồn"]/data_nhapxuatton["Tổng tồn"],0

                                            )
)

In [5]:
#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)
 

In [6]:
# Gộp dữ liệu
data_nhap_ban=data_nhapxuatton.merge(data_danhmuchanghoa[["ID","Hãng"]],on="ID").merge(data_banhang_latest[["ID","Ngày bán","Giá bán giảm"]],on='ID').merge(data_giaban,on="ID")

# TÍnh số tháng tồn trong kho
data_nhap_ban["Số tháng tồn trong kho"]=(363/30)/data_nhap_ban["Giá vốn"]/(np.where(
    # Nếu Tồn dương và đầu kỳ dương thì lấy Giá vốn / (TB Giá trị đầu kỳ + Cuối kỳ)
                    (data_nhap_ban["Giá trị đầu kỳ"]>0) & (data_nhap_ban["Giá trị tồn"]>0),
                               np.mean([data_nhap_ban["Tổng đầu kỳ"],data_nhap_ban["Tổng tồn"]])/
                                np.mean([data_nhap_ban["Giá trị đầu kỳ"],data_nhap_ban["Giá trị tồn"]]),-999))

# Show kết quả
data_nhap_ban_filter=data_nhap_ban[["ID","Tên vật tư","Hãng","Giá vốn","Giá bán","Tổng đầu kỳ","Tổng nhập","Tổng xuất","Tổng tồn", "Giá trị đầu kỳ", "Giá trị nhập" ,"Giá trị xuất", "Giá trị tồn","Số tháng tồn trong kho"]]
data_nhap_ban_filter=data_nhap_ban_filter.query("`Giá vốn`>0 & `Tổng tồn` >0 ").sort_values(by=["Số tháng tồn trong kho"], ascending=False).reset_index(drop=True)

In [7]:
data_MITSUBISHI_6M=data_nhap_ban_filter.query("`Hãng` == 'MITSUBISHI' & `Số tháng tồn trong kho`>6").sort_values(by="Số tháng tồn trong kho",ascending=False).reset_index(drop=True)

In [9]:
data_input

,Ready Items List of VAPC,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Update,2022-08-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Act,NaN,NaN,NaN,NaN,NaN
2,No.,Brand,Order Conf. Date,Order Ref.,PO Number,Part No.,Substitution,Description,Order Qty,FOB Price (USD/Pcs),Total Order Amount,Agt 22 Ready Stock,Số lượng PO của mình,Số lượng chênh lệch,Note
3,IN 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Mitsubishi,2021-03-17 00:00:00,NaN,NaN,1375A444,NaN,"TANK ASSY,RADIATOR CONDENSER",1,7.71,7.71,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,NaN,NaN,NaN,NaN,NaN,04495-YZZZ2,NaN,NaN,300,16.49,NaN,NaN,NaN,NaN,Hàng order không nằm trong back order của NCC
57,NaN,NaN,NaN,NaN,NaN,11213-BZ011,NaN,NaN,2,4.89,NaN,NaN,NaN,NaN,Hàng order không nằm trong back order của NCC
58,NaN,NaN,NaN,NaN,NaN,168000-5470ND AC,NaN,NaN,20,33.94,NaN,NaN,NaN,NaN,Hàng order không nằm trong back order của NCC
59,NaN,NaN,NaN,NaN,NaN,263500-6170ND AC,NaN,NaN,10,15.64,NaN,NaN,NaN,NaN,Hàng order không nằm trong back order của NCC


In [8]:
data_MITSUBISHI_6M

,ID,Tên vật tư,Hãng,Giá vốn,Giá bán,Tổng đầu kỳ,Tổng nhập,Tổng xuất,Tổng tồn,Giá trị đầu kỳ,Giá trị nhập,Giá trị xuất,Giá trị tồn,Số tháng tồn trong kho
0,MR150094,Cao su cân bằng trước lỗ 30,MITSUBISHI,27729.450000,45000.0,6,0,2,4,158454.0,0,52818.0,105636.0,135.492325
1,MB338595,Cao su cân bằng trước lỗ 24,MITSUBISHI,39371.325000,45000.0,12,2,4,10,449959.0,74993,149986.0,374966.0,95.428022
2,1370B272,Cút nước trên Xpander 19-21,MITSUBISHI,65332.050000,95000.0,3,0,1,2,186664.0,0,62221.0,124443.0,57.508186
3,8355A092,Đèn phản quang badersoc sau RH Xpander 19-21,MITSUBISHI,101600.316176,110000.0,39,20,34,25,3743690.0,1954992,3289915.0,2408767.0,36.979488
4,8355A091,Đèn phản quang badersoc sau LH Xpander 19-21,MITSUBISHI,101613.431818,110000.0,39,21,33,27,3743690.0,2051343,3193565.0,2601468.0,36.974715
5,1370B271,Cút nước dưới Xpander 19-21,MITSUBISHI,106620.150000,140000.0,5,0,1,4,507717.0,0,101543.0,406174.0,35.238439
6,6410D211,Tai cài badesoc sau LH Xpander 19-21,MITSUBISHI,137465.300000,170000.0,10,0,3,7,1309193.0,0,392758.0,916435.0,27.331462
7,6410D212,Tai cài badesoc sau RH Xpander 19-21,MITSUBISHI,137465.300000,170000.0,10,0,6,4,1309193.0,0,785516.0,523677.0,27.331462
8,7632D398,Mặt gương RH Xpander 19-21,MITSUBISHI,180454.050000,220000.0,10,0,9,1,1718605.0,0,1546749.0,171856.0,20.820412
9,1340A181,Dây tổng Mitsubishi Xpander 5PK1266,MITSUBISHI,188172.891667,300000.0,25,55,36,44,3887762.0,11098990,6451642.0,8535110.0,19.966360


In [ ]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Lọc tồn theo hãng/Export.xlsx') as writer:
    # data_mitsubishi.to_excel(writer, sheet_name='MITSUBISHI')
    data_nhap_ban.to_excel(writer,sheet_name="Nhap Ban")
    data_nhap_ban_filter.to_excel(writer,sheet_name="Filter")
    data_MITSUBISHI_6M.to_excel(writer,sheet_name="MITSUBISHI_6Thang")